In [1]:
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
m1 = Model('vrptw')
customers = 10
minimum_vehicles_required = 0
final_path_list = []


Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-25


In [2]:
cust_dict = {}
for i in range(customers):
    cust_dict[i] = i


In [3]:
cust_dict

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [4]:
def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw




timeWindows = []
for i in range(0,customers):
    st = random.randint(1,2)*10
    et = random.randint(1,3)*10 + st
    tw = [st,et]
    timeWindows.append(tw)

print("time Windows ", timeWindows)
print()

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))

print("discreteTimeWindows", discreteTimeWindows)
print()


timeMatrix = []
for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            timeMatrix[i][j] = random.randint(1,10)*10
            timeMatrix[j][i] = timeMatrix[i][j]

print("timeMatrix", timeMatrix)
print()




time Windows  [[20, 50], [20, 50], [20, 50], [20, 50], [10, 20], [10, 30], [10, 40], [20, 30], [20, 30], [20, 40]]

discreteTimeWindows [[20, 30, 40, 50], [20, 30, 40, 50], [20, 30, 40, 50], [20, 30, 40, 50], [10, 20], [10, 20, 30], [10, 20, 30, 40], [20, 30], [20, 30], [20, 30, 40]]

timeMatrix [[0, 90, 10, 10, 20, 80, 60, 60, 90, 70], [90, 0, 60, 10, 80, 10, 50, 100, 50, 20], [10, 60, 0, 50, 90, 40, 10, 60, 30, 70], [10, 10, 50, 0, 40, 90, 60, 90, 10, 90], [20, 80, 90, 40, 0, 80, 10, 80, 100, 80], [80, 10, 40, 90, 80, 0, 50, 40, 20, 20], [60, 50, 10, 60, 10, 50, 0, 50, 70, 80], [60, 100, 60, 90, 80, 40, 50, 0, 100, 60], [90, 50, 30, 10, 100, 20, 70, 100, 0, 20], [70, 20, 70, 90, 80, 20, 80, 60, 20, 0]]



In [5]:
discreteTimeWindows

[[20, 30, 40, 50],
 [20, 30, 40, 50],
 [20, 30, 40, 50],
 [20, 30, 40, 50],
 [10, 20],
 [10, 20, 30],
 [10, 20, 30, 40],
 [20, 30],
 [20, 30],
 [20, 30, 40]]

In [6]:
while(customers>0):
    count = 0
    nodeCount = []  # assigns a number to node using count and stores its value

    for i in range(0, len(discreteTimeWindows)):
        c = []
        for j in range(0, len(discreteTimeWindows[i])):
            c.append(count+1)
            count += 1
        nodeCount.append(c)

    # print("Node Count ", nodeCount)
    # print()
    m1 = Model('vrptw')
    edgeFlows = {}
    edgeNumber = 0
    x = [] 
    edges = {}
    edgesList = []  # stores edge number going out of ith node
    edgesList.append([])  # for s
    # solver = pywraplp.Solver.CreateSolver('GLOP')
    # GRAPH CONSTRUCTION

    # out of s to starting of each customer
    e = []
    for i in range(0, len(nodeCount)):
    #     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    #     m1.update()
        edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
        edgeFlows[edgeNumber] = (0, nodeCount[i][0])
        e.append(edgeNumber)
        edgeNumber += 1
    edgesList[0] = e

    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])-1):
            e = []
    #         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    #         m1.update()
            edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
            edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
            # add edge number in edge list
            e.append(edgeNumber)
            edgesList.append(e)
            edgeNumber += 1
        edgesList.append([])
    print("Edges List:", edgesList)

        
    for i in range(len(discreteTimeWindows)):
        tu = nodeCount[i][-1]
        start_utime = discreteTimeWindows[i][0]
        end_utime = discreteTimeWindows[i][-1]

        # stores edges reachable from u
        for k in range(0, len(discreteTimeWindows)):
            sv = nodeCount[k][0]

            start_vtime = discreteTimeWindows[k][0]
            end_vtime = discreteTimeWindows[k][-1]


            if(i != k and start_utime + timeMatrix[i][k] > end_vtime):
                print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)

                # not reachable

            elif(i != k and start_vtime > end_utime + timeMatrix[i][k]):
                # edge bw (u,tu) -> (v,sv)
                print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
                e = []
    #             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                
                edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
                edgeFlows[edgeNumber] = (tu, sv)
                print(tu, "->", sv)
                e.append(edgeNumber)
                edgesList[tu].extend(e)
                edgeNumber += 1


            elif(i != k and start_vtime == end_utime + timeMatrix[i][k]):
                print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
                e = []
    #             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                

                edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
                print(tu, "->", sv)
                edgeFlows[edgeNumber] = (tu, sv)
                e.append(edgeNumber)
                edgesList[tu].extend(e)
                edgeNumber += 1


            elif (i != k and start_vtime < end_utime + timeMatrix[i][k]):
            
                firstReachableNode_index = 0  # first such that node >= sv
                for j in range(0, len(discreteTimeWindows[i])):
                    if(timeMatrix[i][k] + discreteTimeWindows[i][j] >= start_vtime):
                        firstReachableNode_index = j
                        break
                # now starting from firstReachableNode make edges until feasible
                for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                    e = []
                    reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]
                    for r in range(0, len(discreteTimeWindows[k])):
                        if(discreteTimeWindows[k][r] >= reachingTime):
                            print(edgesList[nodeCount[i][l]])
                            print(nodeCount[i][l], "->", nodeCount[k][r])
                            # edge bw these two
    #                         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                            
                            edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                            edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])

                            e.append(edgeNumber)
                            edgesList[nodeCount[i][l]].extend(e)
                            print("hdhdh", edgesList[nodeCount[i][l]])
                            edgeNumber += 1
                            break




    print('edge list b4 s t', edgesList)
    print()

    # into t (except s)
    e = []
    for i in range(0, len(nodeCount)):
    #     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
        edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
        edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
        # add edge number in edge list
        edgesList[nodeCount[i][-1]].append(edgeNumber)
        edgeNumber += 1
    edgesList.append([])


    print("Node Count after s and t:", nodeCount)
    print()

    # GRAPH Construction OVER
    # if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer


    offset = edgeNumber

    print('edge list', edgesList)
    print()
    print("edges", edges)
    print()
    print('edge number', edgeNumber)
    print()

    # print("edge flows:", edgeFlows)

    flows = [0]*customers
    for i in range(customers):
        flows[i] = edgeFlows.copy()
        keys = edgeFlows.keys()
        for j in keys:  # stores key values of edgeflows i.e 0 1 2
            # all x[] > offset are for flow variables
    #         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
        
            # remove key and replace it by edge
            t = flows[i].pop(j)
            flows[i][edgeNumber] = t
            edgeNumber += 1

    # print("flows", flows)
    noc = 0
    # # contraint for outgoing flow for a customer
    for i in range(0, len(nodeCount)):  # choose one list from nodeCount
    #     contraint = [0] * (edgeNumber)
        coeff = [] # store where coeff = 1
        for j in range(0, len(nodeCount[i])):  # se
            node = nodeCount[i][j]
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                if(v not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        coeff.append(x[e])
        
        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1



    incomingEdgeList = []
    for i in range(0, len(edgesList)):
        incomingEdgeList.append([])  # adding lists in a list
    for i in edges:
        v = edges[i][1]  # u -> v

        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

    print("incoming edge list: ", incomingEdgeList)
    print()

    #  constraint for incoming for a customer
    for i in range(0, len(nodeCount)):
        coeff = []
        contraint = [0] * (edgeNumber)  # indicates number of edges
        for j in range(0, len(nodeCount[i])):
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                if(u not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        coeff.append(x[e])
        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1
        
    # constraint for a node incoming = outgoing
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            coeff = []
            contraint = [0] * (edgeNumber)
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                contraint[e] = 1
                coeff.append(x[e])
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                contraint[e] = -1
                coeff.append(-x[e])
            m1.addConstr(( sum(coeff) == 0 ))
            noc+=1  
    # checks wheter an edge(X['1']) returns back to the same customer
    def checkReturningEdges(edge,nodeCount,edges,customeridx,edgelist): 
        (u,v) = edges[edge]
        if(v == -1):
            return False # not returning
        if(v in nodeCount[customeridx]):
            return True
        # search all outgoing edges from v and if even single one of them returns back return true else false
        for key in edges:
            if(edges[key][0] == v):    # key is of the form 'x[i]'
                if(checkReturningEdges(key,nodeCount,edges,customeridx,edgelist)):
                    return True
        return False
                    
                
            
    print('done')
    # #  0<= xij - fij    f<x => 0<=x-f
    for j in range(len(flows)):
        diff = offset*j+offset
        for k in flows[j].keys():
            coeff = [] 
            coeff.append(-x[k])
            coeff.append(x[k-diff])
            m1.addConstr(( sum(coeff) >= 0 ))
            
    print('done')
    #         # print(contraint," >= 0")

    ##constraint for outgoing == 1
    for c in range(customers):
        addn = offset*(c+1)
        
        for i in range(0, len(nodeCount)):
            # indicates number of edges
            coeff = []
            for j in range(0, len(nodeCount[i])):
                node = nodeCount[i][j]
                for k in range(0, len(edgesList[node])):
                    e = edgesList[node][k]
                    u = edges['x[%i]' % e][0]
                    v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                    if(v not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        edge = 'x[%i]' % e
                        node = edges[edge][0]
    #                     print("node:",node)
                        for j in range(0,len(nodeCount)):
                                if(node in nodeCount[j]):
                                        customeridx = j
                                        break    
    #                     print("Customer:",customeridx)
                        if(checkReturningEdges(edge,nodeCount,edges,customeridx,edgesList)==False):
                            coeff.append(x[e+addn])
            m1.addConstr(( sum(coeff) == 1 ))
                        

                



    # # constraint for incoming for a particular customer = 1
    # print("CONSTRAINT for incoming = 1 ")
    # for c in range(customers):
    #     addn = offset*(c+1)
        
    #     for i in range(0, len(nodeCount)):
    #         # indicates number of edges
    #         coeff = []
    #         for j in range(0, len(nodeCount[i])):
    #             node = nodeCount[i][j]
    #             for k in range(0, len(incomingEdgeList[node])):
    #                 e = incomingEdgeList[node][k]
    #                 u = edges['x[%i]' % e][0]
    #                 v = edges['x[%i]' % e][1]
    #                 # print("u", u, "v", v)
    #                 if(u not in nodeCount[i]):
    #                     # print('hello')
    #                     # print("u", u, "v", v)
    #                     coeff.append(x[e+addn])
    #         m1.addConstr(( sum(coeff) == 1 ))
                        
                        


    # # flow conservation for node

    for c in range(customers):
        addn = offset*c+offset
        for i in range(0, len(nodeCount)):
            for j in range(0, len(nodeCount[i])):
                coeff = []
                node = nodeCount[i][j]
                for k in range(0, len(incomingEdgeList[node])):
                    e = incomingEdgeList[node][k]
                    coeff.append(x[e+addn])
                for k in range(0, len(edgesList[node])):
                    e = edgesList[node][k]
                    coeff.append(-x[e+addn])
                m1.addConstr(( sum(coeff) == 0 ))
    objectivefun = []
    for i in range(0, len(edgesList[0])):
        objectivefun.append(x[edgesList[0][i]])
    obj = sum(objectivefun)
    m1.setObjective(obj,GRB.MINIMIZE)
    
        
    m1.optimize()
    print("Runtime is :",m1.Runtime)
    print("objective value : ",m1.objVal)
    for v in m1.getVars():
        print('%s: %g' % (v.varName,v.x))
    print("Number of Constraints :",noc)
    print("Runtime is :",m1.Runtime)
    ans = {}
    for v in m1.getVars()[:offset]:
        if (v.x > 0):
    #         print(v.varName[2:-1])
            ans[(v.varName[2:-1])]=v.x
    ans
    edges
    new_edges = {}   # stores only those edges which are required
    for key in edges:
        if((key[2:-1]) in ans):
            new_edges[(key[2:-1])] = edges[key]
    new_edges
    graphForpaths = []
    for i in range(len(edgesList)): # edgesList contains outgoing from each node
        l = []
        for key in new_edges:
            if(new_edges[key][0] == i):
                if(new_edges[key][1] == -1):
                    l.append(len(edgesList)-1)
                else:
                    l.append(new_edges[key][1])
        graphForpaths.append(l)
    graphForpaths    

    from typing import List

    def allPathsSourceTarget(graph: List[List[int]]) -> List[List[int]]:
        stack = [[0]]
        paths = []
        while stack:
            path = stack.pop()
            node = path[-1]
            if node == len(graph) - 1:
                paths.append(path)
                continue
                
            for new_node in graph[node]:
                stack.append((path + [new_node]))
                        
                
        return paths
    paths = allPathsSourceTarget(graphForpaths)
    for i in range(len(paths)):
        paths[i][-1] = -1
    print(paths)
    print(offset)
    print(nodeCount)
    def distinctCustomer(path,nodeCount):
        customer = 0
        visited = [] # stores index of visited customer
        for i in range(1,len(path)-1):
            node = path[i]
            for j in range(0,len(nodeCount)):
                if(node in nodeCount[j]):
                    if(j not in visited):
                        visited.append(j)
                        customer+=1
        return customer,visited
    # distinctCustomer(paths[2],nodeCount)
    def getmin(path,ans,new_edges):
        for i in range(0,len(path)-1):
            edge = (path[i],path[i+1])
            minVal = 99 # any number > 1
            for key in new_edges:
                if(new_edges[key] == edge):
                    minVal = min(minVal,ans[key])
        return minVal
                    
            
    getmin(paths[0],ans,new_edges)

    # getting the fatest * longest edge, if this number is same of 2 or more paths, then the first path is picked
    def get_best_path(paths,nodeCount, ans,new_edges):
        li = []
        best_value = 0
        best_path = 0
        for i in range(len(paths)):
            l = []
            distinct_cust,visited = distinctCustomer(paths[i], nodeCount)
            min_value = getmin(paths[i],ans,new_edges)
            value = distinct_cust * min_value
            if value>best_value:
                best_value = value
                best_path = i
                
            l.append(distinct_cust)
            l.append(min_value)
            li.append(l)
        
        distinct_cust_final,final_visited = distinctCustomer(paths[best_path], nodeCount)
        return li , best_value, best_path,final_visited
    
    li,best_value, best_path, final_visited = get_best_path(paths,nodeCount, ans,new_edges)
    li
    best_value
    best_path
    
    # so the best path is the path at index of best_path in paths
    customers_served = distinctCustomer(paths[best_path], nodeCount)
    customers_served
    
    # distinct_cust_final,final_visited = distinctCustomer(paths[best_path], nodeCount)
    final_visited = sorted(final_visited)
    final_visited
    discreteTimeWindows 
    discreteTimeWindows2 = []
    for i in range(len(discreteTimeWindows)):
        if i not in final_visited:
            discreteTimeWindows2.append(discreteTimeWindows[i])

    nodeCount 
    nodeCount2 = []
    for i in range(len(nodeCount)):
        if i not in final_visited:
            nodeCount2.append(nodeCount[i])

    timeMatrix2 = []
    for i in range(len(timeMatrix)):
        if i not in final_visited:
            l = []
            for j in range(len(timeMatrix[i])):
                if j not in final_visited:
                    l.append(timeMatrix[i][j])
            timeMatrix2.append(l)

    

    discreteTimeWindows = discreteTimeWindows2
    nodeCount = nodeCount2
    timeMatrix = timeMatrix2

    


 
    for i in final_visited:
            for key, value in cust_dict.items():
                if i == value:
                    cust_dict[key]= -(minimum_vehicles_required+ 1)

    a = 0
    for i in range(len(cust_dict)):
        if cust_dict[i]>= 0 :
            cust_dict[i] = a
            a+=1

    final_str = str()
    final_str = final_str + 'Source -> '

    for i in range(len(cust_dict)):
            if cust_dict[i] == -(minimum_vehicles_required+1):
                node = i
                final_str = final_str + ''.join(str(node)+ ' -> ')
        
    final_str = final_str+' Sink '

    final_path_list.append(final_str)
    
    









    minimum_vehicles_required+=1
    customers = customers - len(final_visited)
    print(cust_dict)
    
    
    


Edges List: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10], [11], [12], [], [13], [14], [15], [], [16], [17], [18], [], [19], [20], [21], [], [22], [], [23], [24], [], [25], [26], [27], [], [28], [], [29], [], [30], [31], []]
customer 0 to 1 start_utime  20  end_utime  50  start_vtime 20  end_vtime 50
[10]
1 -> 10
hdhdh [10, 32]
[11]
2 -> 11
hdhdh [11, 33]
[12]
3 -> 12
hdhdh [12, 34]
[10, 32]
1 -> 14
hdhdh [10, 32, 35]
[11, 33]
2 -> 15
hdhdh [11, 33, 36]
[12, 34]
3 -> 16
hdhdh [12, 34, 37]
customer 0 to 4 start_utime  20  end_utime  50  start_vtime 10  end_vtime 20
customer 0 to 5 start_utime  20  end_utime  50  start_vtime 10  end_vtime 30
customer 0 to 6 start_utime  20  end_utime  50  start_vtime 10  end_vtime 40
customer 0 to 7 start_utime  20  end_utime  50  start_vtime 20  end_vtime 30
customer 0 to 8 start_utime  20  end_utime  50  start_vtime 20  end_vtime 30
customer 0 to 9 start_utime  20  end_utime  50  start_vtime 20  end_vtime 40
customer 1 to 0 start_utime  20  end_utime  50  star

In [7]:
minimum_vehicles_required

4

In [8]:
customers_left = customers
customers_left

0

In [9]:
final_path_list

['Source -> 0 -> 2 -> 4 -> 6 ->  Sink ',
 'Source -> 1 -> 3 -> 5 -> 8 ->  Sink ',
 'Source -> 9 ->  Sink ',
 'Source -> 7 ->  Sink ']